 # Demonstrate How to Use Crawford's Single-Case Methodologies in Python

In [1]:
# Import Python packages
import pandas as pd
from pathlib import Path

# Import all functions for single-case methods
from crawfordtests.singlecase_t import crawford_t
from crawfordtests.singlecase_dissocs import crawford_udt, crawford_rsdt

 ## Load Sample Data
 Load sample data of a patient and controls

 The csv file contains data of a face oddity task from a single patient
 and his demographically-matched controls.

 There are two conditions of the face oddity task: different- and same-view conditions.

 The dependent variable/measure is accuracy (proportion of correct trials).

In [2]:
# Load sample data of a patient and controls
cwd = Path.cwd()
filePath = cwd.parent / 'data_singlecase.csv'
data_all = pd.read_csv(filePath)
print(data_all.info())
data_all.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28 entries, 0 to 27
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Subject    28 non-null     object 
 1   Group      28 non-null     object 
 2   Condition  28 non-null     object 
 3   Accuracy   28 non-null     float64
dtypes: float64(1), object(3)
memory usage: 1.0+ KB
None


,Subject,Group,Condition,Accuracy
0,p02,Patient,Diff,0.450
1,p02,Patient,Same,0.950
2,c06,Control,Diff,0.775
3,c07,Control,Diff,0.500
4,c08,Control,Diff,0.975


In [3]:
# Separate patient's data from controls'
data_p = data_all.loc[data_all['Group']=='Patient'] # the patient's data
data_c = data_all.loc[data_all['Group']=='Control'] # all controls' data

 ## Single-Case T-Test
 Testing whether or not the patient's accuracy is significantly different from the control group

In [4]:
_res_df = pd.DataFrame()
# Conduct the single-case t-test for each condition separately
for cond in data_all['Condition'].unique().tolist():
    _tmp = crawford_t(
        case=data_p[data_p['Condition']==cond],
        control=data_c[data_c['Condition']==cond],
        dv='Accuracy'
    )
    _tmp.index = [cond]
    _res_df = pd.concat([_res_df, _tmp], axis=0)
    # print('Crawford T-test of condition', cond+':')
    # print(_tmp, '\n')

print("Crawford's T-test:")
_res_df.round(4)

Crawford's T-test:


,tVal,df,pVal,Zcc,Zcc_95CI,normBelowCase%,normBelowCase%_95CI
Diff,-2.8242,12,0.0153,-2.9308,"[-4.1987, -1.6418]",0.1691,"[0.0046, 6.3204]"
Same,-1.3887,12,0.1902,-1.4412,"[-2.2146, -0.6396]",7.4771,"[1.962, 27.5786]"


 **Summary**
 * The patient's accuracy is significantly lower than the control group in the different-view condition
 * In contrast, there is no difference in accuracy of the same-view condition between the patient and controls

 ## Dissociation Test
 Compare the difference between the patient's performance on two conditions of the same task
 with the distribution of differences in controls.

 * Unstandardized dissociation test (UDT)
 * Revised standardized dissociation test (RSDT)

In [5]:
# UDT
print("Crawford's Unstandardized Dissociation Test (Different-view vs. Same-view):")
crawford_udt(
    case=data_p, control=data_c, 
    dv='Accuracy', subject='Subject',
    cond='Condition', orderCond=['Diff','Same']
).round(4)

Crawford's Unstandardized Dissociation Test (Different-view vs. Same-view):


,tVal,df,pVal,Zdcc
UDT,-2.6216,12,0.0223,-1.2012


In [6]:
# RSDT
print("Crawford's Revised Standardized Dissociation Test (Different-view vs. Same-view):")
crawford_rsdt(
    case=data_p, control=data_c, 
    dv='Accuracy', subject='Subject',
    task='Condition', orderTask=['Diff','Same']
).round(4)

Crawford's Revised Standardized Dissociation Test (Different-view vs. Same-view):


,tVal,df,pVal,Zdcc
RSDT,-1.0953,12,0.2871,-1.2012


 **Summary**
 * The patient's difference in accuracy between the two conditions is different from that of controls, but it is only significant when the scores are not standardized.